In [1]:
import simple_sim.simulation_objects as Objects
import simple_sim.simulation_server as World
import pybullet_data
import os
import simple_sim.simulation_robot as Robots
import simple_sim.simulation_controller as Controls

In [9]:
import numpy as np
from time import sleep

In [2]:
# Start a simulation server
server = World.simulationServer()

In [3]:
# Add a heightmap
heightmap = Objects.SimHeightmap(server, './models/heightmap.obj')

In [4]:
robot = Robots.Robot(server, './models/SimpleWalker/urdf/SimpleWalker.urdf', basePosition = [0, 0, 2])

In [5]:
# Set the gravity
server.setGravity([0, 0, -9.81])

In [6]:
for joint in robot.joints.keys():
    Controls.PositionControl(robot, joint, False)

In [31]:
# Start the simulation
server.startSimulation()

In [26]:
import pybullet

In [29]:
pybullet.setRealTimeSimulation(0)

In [33]:
def control(time):
    return np.clip(np.sin(0.1*time) + 0.5 , -np.pi, np.pi)

In [34]:
t = 0
while t < 10:
    for controller in robot.controller.values():
        controller.setReference(reference = control(t))
    t += 0.1
    sleep(0.5)

In [32]:
testy = robot.controller[1]
testy

In [21]:
def slidingMode(joint, reference):
    # Domain of non switching
    eps = 0.2
    error = # Get Joint Pos - reference
        
    if error < -eps:
        return 10
    elif error > eps:
        return -10
    else: 
        return 0